In [1]:
import edward as ed
import tensorflow as tf

from edward.models import Normal, Categorical
from scipy.misc import imsave
from tensorflow.contrib import slim
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
M = 100 # batch size during training

In [ ]:
M =  # minibatch size
H =  # hidden layer size
D =  # number of features
K =  # number of class labels

W_0 = Normal(mu=tf.zeros([D, H]), sigma=tf.ones([D, H]))
W_1 = Normal(mu=tf.zeros([H, K]), sigma=tf.ones([H, K]))
b_0 = Normal(mu=tf.zeros(H), sigma=tf.ones(L))
b_1 = Normal(mu=tf.zeros(K), sigma=tf.ones(K))

def neural_network(x):
    h = tf.nn.tanh(tf.matmul(x, W_0) + b_0)
    h = tf.matmul(h, W_1) + b_1
    return tf.reshape(h, [-1])

x = ed.placeholder(tf.float32, [M, D])
y = Categorical(logits=neural_network(x))